# Summary 
Tests/demos the subsetted geometry output from the `ArbitrayGeomPHHCalc.ipynb` notebook. 
Adds basic stats from a single quarter and makes a sample map. 

In [ ]:
import sys
sys.path.append("..")

%load_ext autoreload
%autoreload 1
%aimport src.datasets.joins
%aimport src.datasets.loading.statcan

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
from src.datasets.loading import statcan
from src.datasets.loading import ookla
from src.datasets import overlays

import statsmodels as sm
# import statsmodels.stats.weightstats
from scipy.stats import lognorm

import src.config
from pathlib import Path
import geopandas as gp
import src.datasets.joins as joins

In [ ]:
CRS = 'EPSG:4326'

In [ ]:
popctrs = statcan.boundary('population_centres')

In [ ]:
popctrs.crs

In [ ]:
derived_geometry = (Path(src.config.DATA_DIRECTORY) / 'processed' / 'geometries').resolve()

speed_data = gp.read_file(derived_geometry / 'hexagons_w_dissolved_smaller_popctrs.geojson')#, driver='GeoJSON')
# #speed_data = speed_data.to_crs(CRS)
# speed_data.crs = popctrs.crs #assign crs b/c geojson assumes GPS/epsg:4326
speed_data = speed_data.to_crs(popctrs.crs) #now encoded in GPS-Lat/Lon, but we should convert it back

In [ ]:
tiles = ookla.canada_speed_tiles()#.loc[lambda s:(s.year==2022 ) & (s.conn_type=='fixed')]
tiles = tiles.to_crs(CRS)

In [ ]:
speed_data = joins.add_simple_stats(speed_data, tiles.loc[lambda s:(s.year==2022) & (s.conn_type=='fixed')], 'HEXUID_PCPUID')
speed_data = joins.add_50_10_stats(speed_data, tiles.loc[lambda s:(s.year==2022) & (s.conn_type=='fixed')], 'HEXUID_PCPUID')

In [ ]:
speed_data

In [ ]:
unused_columns = ['pc_area','hex_area','pc_frac', 'hex_frac']
for col in unused_columns:
    del speed_data[col]

In [ ]:
speed_data['ookla_50_10_percentile'] = speed_data.apply(lambda s:min(s['50_down_percentile'], s['10_up_percentile']), axis=1)

In [ ]:
speed_data.columns

In [ ]:
xmin, ymin, xmax, ymax = popctrs.loc[lambda s:s.PCNAME=="Lloydminster"].buffer(90_000).total_bounds
speed_data.cx[xmin:xmax,ymin:ymax].loc[lambda s:(s.Pop2016>0) | (s.avg_d_kbps > 0)].explore(
    'ookla_50_10_percentile',scheme='equalinterval', k = 4, 
    tooltip=['HEXUID_PCPUID','PCNAME','Pop2016','Pop_Avail_50_10','ookla_50_10_percentile'],
    popup=['HEXUID_PCPUID','PCNAME','min_d_kbps','avg_d_kbps','max_d_kbps','min_u_kbps','avg_u_kbps','max_u_kbps','Pop2016','tests','unique_devices','Pop_Avail_50_10','ookla_50_10_percentile','50_down_percentile','10_up_percentile']
    )